In [117]:
import h5py
import vedo
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage

from histalign.backend.ccf.paths import get_structure_mask_path
from histalign.backend.io import load_volume
from histalign.backend.models import Resolution
from histalign.backend.preprocessing import normalise_array
from histalign.backend.workspace import Volume

vedo.settings.default_backend = "vtk"

In [118]:
def show(
    meshes: vedo.CommonVisual | list[vedo.CommonVisual], camera: dict | None = None
) -> vedo.Plotter | None:
    plotter = vedo.Plotter(axes=4)

    if not isinstance(meshes, list):
        meshes = [meshes]

    for mesh in meshes:
        plotter += mesh

    plotter.show(interactive=False, camera=camera).interactive().close()


def imshow(
    image: np.ndarray,
    title: str = "",
    figsize: tuple[int, int] | None = None,
    save: bool = False,
) -> None:
    plt.figure(figsize=figsize)
    plt.suptitle(title)

    plt.axis(False)
    plt.imshow(image)

    if save:
        plt.imsave(f"{title}.png", image)
    plt.show()

In [119]:
path = "/home/ediun/.local/share/histalign/interpolated_volumes/a867dab107ba5d7654de284a729d49fe_multiquadric_27_1_0_0.h5"
volume = h5py.File(path)["array"][:]

mask = load_volume(
    get_structure_mask_path("root", Resolution.MICRONS_100), return_raw_array=True
)

# volume = volume[..., : volume.shape[2] // 2]
# mask = mask[..., : mask.shape[2] // 2]

volume_shape = volume.shape
shape = volume.shape[2:] + volume.shape[1:]

In [120]:
volume = ndimage.gaussian_filter(
    volume,
    sigma=5,
    radius=20,
)

volume = normalise_array(volume, dtype=np.uint8)
volume = np.digitize(volume, np.linspace(0, 256, 25))

volume = np.where(mask, volume, 0)

In [121]:
camera = {
    "focal_point": (np.array(shape) - 1) // 2,
    "pos": ((np.array(shape) - 1) // 2) - [0, 0, 500],
    "viewup": (0, -1, 0),
}

volume = vedo.Volume(volume)
plotter = vedo.Plotter(axes=4)


def slice_volume(widget, event):
    global plotter

    mesh = vedo.Image(normalise_array(volume.tonumpy()[int(widget.value)], np.uint8))

    plotter.clear()
    plotter += mesh


plotter.add_slider(slice_volume, xmin=0, xmax=volume_shape[0] - 1, value=70)
plotter += vedo.Image(normalise_array(volume.tonumpy()[70], np.uint8), channels=1)

plotter.show(interactive=False, camera=camera).interactive().close()

/home/ediun/git/histalign/src/histalign/backend/preprocessing/__init__.py:37: RuntimeWarning: invalid value encountered in divide
  return (maximum * (array - np.min(array)) / np.ptp(array)).astype(dtype)
